In [2]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
#import tim
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine
#https://chmodi.tistory.com/114

In [3]:
import psycopg2
from sshtunnel import SSHTunnelForwarder

try:
    tunnel= SSHTunnelForwarder(
         ('210.91.223.248', 35430),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="medicalai",
         ssh_password="medicalai731!", 
         remote_bind_address=('localhost', 35432))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': 'mai_admin',
             'password': 'admin1q2w!@',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()


tunnel connected
database connected


In [4]:
# curs.execute("select * from mimiciv_derived.vitalsign;")
# columns_name = [desc[0] for desc in curs.description]
# vitalsign = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [5]:
# vitalsign.to_csv('../outputs/vitalsign.csv', index=False, encoding='utf-8-sig')
vitalsign = pd.read_csv('../outputs/vitalsign.csv')

In [6]:
vitalsign

,subject_id,stay_id,charttime,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,temperature,temperature_site,spo2,glucose
0,10000032,39553978,2180-07-23 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.06,Oral,NaN,NaN
1,10000032,39553978,2180-07-23 14:11:00,NaN,84.0,48.0,56.0,84.0,48.0,56.0,NaN,NaN,NaN,NaN,NaN
2,10000032,39553978,2180-07-23 14:12:00,91.0,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN
3,10000032,39553978,2180-07-23 14:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.0,NaN
4,10000032,39553978,2180-07-23 14:30:00,93.0,95.0,59.0,67.0,95.0,59.0,67.0,21.0,NaN,NaN,97.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9745495,19999987,36195440,2145-11-04 19:31:00,NaN,163.0,120.0,131.0,163.0,120.0,131.0,NaN,NaN,NaN,NaN,NaN
9745496,19999987,36195440,2145-11-04 19:37:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN
9745497,19999987,36195440,2145-11-04 20:00:00,92.0,NaN,NaN,NaN,NaN,NaN,NaN,23.0,37.83,Oral,100.0,NaN
9745498,19999987,36195440,2145-11-04 21:00:00,87.0,NaN,NaN,NaN,NaN,NaN,NaN,24.0,37.00,Oral,98.0,NaN


In [7]:
subjectlist1 = pd.read_csv('./data/subjectlist1.csv', parse_dates=['admittime','intubationtime','dischtime', 'deathtime','extubationtime','reintubation_eventtime'])
sub1list = subjectlist1 >> select('subject_id','hadm_id','stay_id')

In [8]:
sub1_vitals = sub1list >> left_join(vitalsign, by=['subject_id','stay_id'])

In [9]:
# sub1_vitals.to_csv('a../outputs/sub1_vitals.csv', index=False, encoding='utf-8-sig')

In [10]:
sub1_vitals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3391280 entries, 0 to 3391279
Data columns (total 16 columns):
 #   Column            Dtype  
---  ------            -----  
 0   subject_id        int64  
 1   hadm_id           int64  
 2   stay_id           int64  
 3   charttime         object 
 4   heart_rate        float64
 5   sbp               float64
 6   dbp               float64
 7   mbp               float64
 8   sbp_ni            float64
 9   dbp_ni            float64
 10  mbp_ni            float64
 11  resp_rate         float64
 12  temperature       float64
 13  temperature_site  object 
 14  spo2              float64
 15  glucose           float64
dtypes: float64(11), int64(3), object(2)
memory usage: 414.0+ MB
